In [1]:
import sys
sys.path.append('../')
from setting import config_read

In [2]:
import pandas as pd
import owlready2
from mapping import ResourceMapper
from elasticsearch import Elasticsearch
from graph import QueryGraphGenerator
import itertools

* Owlready2 * Warning: optimized Cython parser module 'owlready2_optimized' is not available, defaulting to slower Python implementation




In [3]:
config = config_read('../')

# Elasticsearch
server_ip = config['elasticsearch']['ip']
index_name = config['elasticsearch']['name']
es = Elasticsearch(server_ip)
rm = ResourceMapper(config)

In [18]:
# user_input = '용감한 형제가 작곡한 노래의 장르는?'
user_input = '뱅뱅뱅 부른 음악그룹의 소속사'
resource_combinations = rm.process(user_input)

In [19]:
def get_type(resource):
    search_query = {"query":{"term":{"URI.keyword": resource}}}
    result = es.search(index=index_name, body=search_query)
    return result['hits']['hits'][0]['_source']['Type']

In [20]:
conceptual_graph = []
for rc in resource_combinations:
    # rightmost element is an edge
    if get_type(rc[-1]) in ['T_dp', 'T_op']:
        rc = list(rc)
        rc.append('owl:Thing')

    conceptual_arc_list = []

    for i in range(len(rc)-1):
        conceptual_arc = []
        s_type = get_type(rc[i])

        if s_type in ['T_dp', 'T_op']:
            continue
        
        p_list = []
        for j in range(i+1, len(rc)):
            o_type = get_type(rc[j])
            if o_type in ['T_dp', 'T_op']:
                p_list.append(rc[j])
            elif s_type == 'T_i' and o_type == 'T_i':
                continue
            else:
                if len(p_list) == 0:
                    conceptual_arc.append((rc[i],'Any P',rc[j]))
                else:
                    for p in p_list:
                        conceptual_arc.append((rc[i], p, rc[j]))
        conceptual_arc_list.append(conceptual_arc)
    
    for cg in list(itertools.product(*conceptual_arc_list)):
        conceptual_graph.append(list(cg))

conceptual_graph

[[('skmo:뱅뱅뱅', 'skmo:isSungBy', 'mo:MusicGroup'),
  ('mo:MusicGroup', 'schema:affiliation', 'owl:Thing')],
 [('skmo:뱅뱅뱅', 'skmo:isSungBy', 'owl:Thing'),
  ('mo:MusicGroup', 'schema:affiliation', 'owl:Thing')],
 [('skmo:뱅뱅뱅', 'schema:affiliation', 'owl:Thing'),
  ('mo:MusicGroup', 'schema:affiliation', 'owl:Thing')],
 [('skmo:뱅뱅뱅_inst', 'skmo:isSungBy', 'mo:MusicGroup'),
  ('mo:MusicGroup', 'schema:affiliation', 'owl:Thing')],
 [('skmo:뱅뱅뱅_inst', 'skmo:isSungBy', 'owl:Thing'),
  ('mo:MusicGroup', 'schema:affiliation', 'owl:Thing')],
 [('skmo:뱅뱅뱅_inst', 'schema:affiliation', 'owl:Thing'),
  ('mo:MusicGroup', 'schema:affiliation', 'owl:Thing')],
 [('skmo:뱅뱅뱅_황치열', 'skmo:isSungBy', 'mo:MusicGroup'),
  ('mo:MusicGroup', 'schema:affiliation', 'owl:Thing')],
 [('skmo:뱅뱅뱅_황치열', 'skmo:isSungBy', 'owl:Thing'),
  ('mo:MusicGroup', 'schema:affiliation', 'owl:Thing')],
 [('skmo:뱅뱅뱅_황치열', 'schema:affiliation', 'owl:Thing'),
  ('mo:MusicGroup', 'schema:affiliation', 'owl:Thing')]]

In [21]:
conceptual_graph = []
for rc in resource_combinations:
    # rightmost element is an edge
    if get_type(rc[-1]) in ['T_dp', 'T_op']:
        rc = list(rc)
        rc.append('owl:Thing')
        
    conceptual_arc = []
    for i in range(len(rc)-1):
        s_type = get_type(rc[i])

        if s_type in ['T_dp', 'T_op']:
            continue
        
        p = 'Any P'
        for j in range(i+1, len(rc)):
            o_type = get_type(rc[j])
            if o_type in ['T_dp', 'T_op']:
                p = rc[j]
            elif s_type == 'T_i' and o_type == 'T_i':
                break
            else:
                conceptual_arc.append((rc[i],p,rc[j]))
                break
    conceptual_graph.append(conceptual_arc)

conceptual_graph

[[('skmo:뱅뱅뱅', 'skmo:isSungBy', 'mo:MusicGroup'),
  ('mo:MusicGroup', 'schema:affiliation', 'owl:Thing')],
 [('skmo:뱅뱅뱅_inst', 'skmo:isSungBy', 'mo:MusicGroup'),
  ('mo:MusicGroup', 'schema:affiliation', 'owl:Thing')],
 [('skmo:뱅뱅뱅_황치열', 'skmo:isSungBy', 'mo:MusicGroup'),
  ('mo:MusicGroup', 'schema:affiliation', 'owl:Thing')]]